### Initialize

In [41]:
from datetime import datetime
import json
import os
import sys

from ipywidgets import Dropdown
import pandas as pd
import requests

from shared import print_response, StopExecution, get_base_url, get_root_path

base_url = get_base_url()
root_path = get_root_path()

### Select example which should be used

In [42]:
# example_path = os.path.join('..', 'examples')
example_path = 'C:\\Users\\I570329\\Desktop\\Datenhaus\\hana-enterprise-search-engine-tests\\demo\\examples'
options = next(os.walk(example_path, topdown=True))[1]
selected = Dropdown(
    options=options,
    description='Example:',
    value = None,
    disabled=False,
)
selected

Dropdown(description='Example:', options=('documents', 'icm2', 'messenger', 'offshore', 'simple'), value=None)

### Create CSON (JSON-format) from CDS file (text format) using the CDS compiler

In [127]:
from json import JSONDecodeError


if selected.value:
    # cds_file = os.path.join(root_path, '..', 'examples', selected.value, 'icm.model.cds')
    cds_file = "C:\\Users\\I570329\\Desktop\\Datenhaus\\hana-enterprise-search-engine-tests\\demo\\examples\\icm2\\model.cds"
    # cson_file = os.path.join(root_path, '..', 'temp', f'{selected.value}.model.json')
    cson_file = "C:\\Users\\I570329\\Desktop\\Datenhaus\\hana-enterprise-search-engine-tests\\demo\\examples\\icm2\\temp\\model.json"
    # data_file = os.path.join(root_path, '..', 'examples', selected.value, 'icm.data.json')
    data_file = "C:\\Users\\I570329\\Desktop\\Datenhaus\\hana-enterprise-search-engine-tests\\demo\\examples\\icm2\\data.json"
    command_line_statement = (f'cds compile {cds_file} --to json > {cson_file}')
    os.system(command_line_statement)
  
    with open(cson_file, encoding='utf-8') as f:
        cson = json.load(f)

    print(f'The following command line statement was executed to compile the example cds file to cson. Filename is temp{os.sep}{selected.value}.model.json):')
    print(command_line_statement)
else:
    print('No example selected')

The following command line statement was executed to compile the example cds file to cson. Filename is temp\icm2.model.json):
cds compile C:\Users\I570329\Desktop\Datenhaus\hana-enterprise-search-engine-tests\demo\examples\icm2\model.cds --to json > C:\Users\I570329\Desktop\Datenhaus\hana-enterprise-search-engine-tests\demo\examples\icm2\temp\model.json


### List existing tenants

In [128]:
r = requests.get(f'{base_url}/v1/tenant')
if r.status_code == 200:
    existing_tenants = [{k:datetime.fromisoformat(v) if k == 'createdAt' else v 
        for k, v in w.items()} for w in r.json()]
    if existing_tenants:
        df = pd.DataFrame.from_dict(existing_tenants)
        df.rename(columns={'name': 'Tenant Name', 'createdAt': 'Created At'}, inplace=True)
        display(df)
    else:
        print('Info: No tenants exist')
else:
    print(f'Error. HTTP response status code {r.status_code}')

Info: No tenants exist


### Create tenant

In [129]:
r = requests.post(f'{base_url}/v1/tenant/{selected.value}Example')
print_response(r)

Success: Tenant 'icm2Example' successfully created


### Check model consistency

In [130]:
r = requests.post(f'{base_url}/v1/deploy/{selected.value}', json=cson\
                    , params={'simulate': True})
print_response(r)

Success: Model is consistent


### Deploy

In [131]:
r = requests.post(f'{base_url}/v1/deploy/{selected.value}Example', json=cson)
print_response(r)

Success: Model successfully deployed


In [126]:
with open(data_file, encoding='utf-8') as p:
    data = json.load(p)

r = requests.post(f'{base_url}/v1/data/{selected.value}Example', json=data)

# process_service_response(package, test, TestType.LOAD_DATA, r)

if r.status_code == 200:

    ids = r.json()

    if ids:

        # object_ids = deepcopy(ids)

        r = requests.post(f'{base_url}/v1/read/{selected.value}Example', json= ids)

        # process_service_response(package, test, TestType.READ_DATA, r)
else:
    print("Error:", r.status_code)        

In [24]:
if selected.value:
    ids = r.json()
    if ids:
        r = requests.post(f'{base_url}/v1/read/{selected.value}Example', json=ids)
        if r.status_code == 200:
            data_read = r.json()
            for typ, obj_list in data_read.items():
                for obj in obj_list:
                    if 'id' in obj:
                        del obj['id']
            if data_read == data:
                print('Data successfully validated')
            else:
                print('Error: Returned data are different to loaded data')
        else:
            print('Error while reading data')

Error: Returned data are different to loaded data
